In [3]:
import matplotlib.pyplot as plt
from aim import Run
import os
from src.utils.ProjectConfiguration import ProjectConfiguration as pc
from aim.storage.context import Context
import pandas as pd
import seaborn as sns
import json, numpy as np
import colormaps as cmaps

In [ ]:
models = [
    ("<path>/Experiments/cholecfFixAbl_none_10_1.0_16_1_1.0_0.99_OctreeNCASegmentation", "OctreeNCA"),
    ("<path>/Experiments/cholec_unet_UNetSegmentation/", "UNet"),
    ("<path>/Experiments/cholec_munet3d_UNet_resnet18_0_minUNet3DSegmentation/", "Resnet18"),
    ("<path>/Experiments/cholec_munet3d_UNet_densenet121_0_minUNet3DSegmentation/", "Densenet"),
    ("<path>/Experiments/cholec_M3dSegmentation/", "M3D-NCA"),
    ("<path>/Experiments/cholec_M3dSegmentation/", "gt"),
]

In [ ]:
test_cases = [
"video01_00080",
"video01_00160",
"video01_00240",
"video01_00400",
"video01_14859",
"video01_14939",
"video01_15019",
"video01_15099",
"video01_16345",
"video01_16425",
"video01_16585",
"video01_28580",
"video01_28660",
"video01_28740",
"video01_28820",
"video01_28900",
"video12_15750",
"video12_15830",
"video12_19500",
"video12_19580",
"video12_19660",
"video12_19740",
"video12_19900",
"video12_19980",
"video17_01563",
"video17_01643",
"video17_01803",
"video17_01963",
"video27_00160",
"video27_00240",
"video27_00400",
"video27_00480",
"video27_00640",
"video52_00000",
"video52_00080",
"video52_00160",
"video52_00240",
"video52_00320",
"video52_00400",
"video52_00480",
"video52_02746",
"video52_02826",
"video52_02906"]

In [ ]:
def get_image(path, sample, gt=False):
    config = json.load(open(os.path.join(path, "config.json"), "r"))
    run_hash = config["experiment.run_hash"]
    exp_name = config["experiment.name"]
    run = Run(run_hash=run_hash, 
                    experiment=exp_name, repo=os.path.join(pc.FILER_BASE_PATH, pc.STUDY_PATH, 'Aim'),
                    read_only=True)
    idx, imgs = run.get_image_sequence(f"test/img/{sample}", Context({})).data.items_list()
    arr = np.array(imgs[0][-1].to_pil_image())
    WIDTH = arr.shape[1] // 3
    img = arr[:,:WIDTH]
    if gt:
        seg = arr[:, 2*WIDTH:3*WIDTH]
    else:
        seg = arr[:, WIDTH:2*WIDTH]
    alpha = np.zeros((img.shape[0], img.shape[1], 4))
    alpha[:,:,:3] = seg
    alpha[:,:,3] = (seg[:,:,0] != 0).astype(np.uint8) * 125
    alpha = alpha.astype(np.uint8)
    return img, alpha

In [ ]:
for path, name in models:
    for case in test_cases:
        if name == "gt":
            img, alpha = get_image(path, case, gt=True)
        else:
            img, alpha = get_image(path, case, gt=False)
        plt.clf()
        plt.imshow(img)
        plt.imshow(alpha)
        plt.axis('off')
        plt.savefig(f"figures/qualitative/cholec/{name}_{case}_pred.png", bbox_inches='tight', pad_inches=0)

In [ ]:
import matplotlib.pyplot as plt

# Colors and labels for the legend
colors = [cmaps.bold[i].colors for i in range(5)]
labels = ['Abdominal wall', 'Liver', 'Fat', 'Grasper', 'Gallbladder']

# Create a figure and axis (optional for just a legend)
fig, ax = plt.subplots()

# Create legend handles with points
handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10) for color in colors]

# Add legend to the plot
ax.legend(handles, labels, title="", loc="upper left", ncol=len(labels), handletextpad=0)

# Show or save the figure
plt.axis('off')  # Turn off axis if it's just the legend
plt.savefig(f"figures/qualitative/cholec/legend.pdf", bbox_inches='tight')
